In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

titanic_path = "C:/Users/weiso131/Desktop/sklearn/titanic_disaster/titanic/train.csv"

titanic_df = pd.read_csv(titanic_path)

titanic_df = titanic_df.sample(n=len(titanic_df))

In [2]:
from sklearn.impute import SimpleImputer

def data_pre_process(keep : list, df : pd.DataFrame):
    new_df = df[keep]
    
    for i in new_df.index:
        if (new_df["Sex"].get(i) == "female"): 
            new_df.at[i ,"Sex"] = 0
        else: 
            new_df.at[i ,"Sex"] = 1

    imputer = SimpleImputer(strategy='median')
    if ("Age" in keep):  new_df['Age'] = imputer.fit_transform(new_df[["Age"]])
    if ("Fare" in keep): new_df['Fare'] = imputer.fit_transform(new_df[["Fare"]])

    
    return new_df

In [3]:
keep = ["Survived", "Pclass", "Sex", "SibSp", "Parch", "Fare", "Age"]
keep_Experimental = ["Survived", "Pclass", "Sex", "SibSp", "Parch"]
titanic_steady_df = data_pre_process(keep, titanic_df)
titanic_Experimental_df = data_pre_process(keep_Experimental, titanic_df)


C:\Users\weiso131\AppData\Local\Temp\ipykernel_10052\1911100202.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if ("Age" in keep):  new_df['Age'] = imputer.fit_transform(new_df[["Age"]])
C:\Users\weiso131\AppData\Local\Temp\ipykernel_10052\1911100202.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if ("Fare" in keep): new_df['Fare'] = imputer.fit_transform(new_df[["Fare"]])


In [4]:
X_steady = titanic_steady_df.drop("Survived", axis=1).values
y_steady = titanic_steady_df["Survived"].values



X_Experimental = titanic_Experimental_df.drop("Survived", axis=1).values
y_Experimental = titanic_Experimental_df["Survived"].values

In [5]:
def cut_block(X : list, y : list, cut=5):
    """
    return X_block, y_block
    """
    n_20 = int(len(X) / cut)
    X_block = []
    y_block = []
    for i in range(cut):
        X_block.append(X[i*n_20:(i+1)*n_20])
        y_block.append(y[i*n_20:(i+1)*n_20])
    return X_block, y_block

def assemble_block(X_block : list, y_block : list, test_i : int, cut=5):
    """
    return:
    x_train, y_train, x_test, y_test
    """
    x_test, y_test = X_block[test_i], y_block[test_i]
    x_train = []
    y_train = []
    for i in range(cut):
        if (i == test_i): continue
        x_train.extend(X_block[i])
        y_train.extend(y_block[i])

    return x_train, y_train, x_test, y_test


In [6]:
def train(x_train : list, y_train : list):
    """
    return rf_clf
    """
    # 創建隨機森林分類器實例
    rf_clf = RandomForestClassifier(n_estimators=1000, random_state=42)

    # 訓練分類器
    rf_clf.fit(x_train, y_train)

    

    return rf_clf
def test(x_train : list, y_train : list, x_test : list, y_test : list, rf_clf):
    prediction_train = rf_clf.predict(x_train)
    print(f"train_acc: {accuracy_score(prediction_train, y_train)}")
    prediction_test = rf_clf.predict(x_test)
    print(f"test_acc: {accuracy_score(prediction_test, y_test)}")

    return accuracy_score(prediction_train, y_train), accuracy_score(prediction_test, y_test)


In [7]:
cut = 10

X_steady_block, y_steady_block = cut_block(X_steady, y_steady, cut=cut)
X_Experimental_block, y_Experimental_block = cut_block(X_Experimental, y_Experimental, cut=cut)  



avg_steady_train = 0
avg_steady_test = 0
avg_Experimental_train = 0
avg_Experimental_test = 0



for i in range(cut):
    x_steady_train, y_steady_train, x_steady_test, y_steady_test = \
        assemble_block(X_steady_block, y_steady_block, i, cut=cut)
    
    x_Experimental_train, y_Experimental_train, x_Experimental_test, y_Experimental_test = \
        assemble_block(X_Experimental_block, y_Experimental_block, i, cut=cut)

    
    rf_clf_steady = train(x_steady_train, y_steady_train)
    print("steady:")
    result = test(x_steady_train, y_steady_train, x_steady_test, y_steady_test, rf_clf_steady)

    avg_steady_train += result[0]
    avg_steady_test += result[1]

    
    rf_clf_Experimental = train(x_Experimental_train, y_Experimental_train)
    print("Experimental:")
    result = test(x_Experimental_train, y_Experimental_train, x_Experimental_test, y_Experimental_test, rf_clf_Experimental)

    avg_Experimental_train += result[0]
    avg_Experimental_test += result[1]

steady:
train_acc: 0.9800249687890137
test_acc: 0.7865168539325843
Experimental:
train_acc: 0.8202247191011236
test_acc: 0.797752808988764
steady:
train_acc: 0.978776529338327
test_acc: 0.8202247191011236
Experimental:
train_acc: 0.8127340823970037
test_acc: 0.8314606741573034
steady:
train_acc: 0.978776529338327
test_acc: 0.8314606741573034
Experimental:
train_acc: 0.8202247191011236
test_acc: 0.7752808988764045
steady:
train_acc: 0.9775280898876404
test_acc: 0.8089887640449438
Experimental:
train_acc: 0.8177278401997503
test_acc: 0.7752808988764045
steady:
train_acc: 0.9837702871410736
test_acc: 0.7865168539325843
Experimental:
train_acc: 0.8302122347066168
test_acc: 0.7078651685393258
steady:
train_acc: 0.9837702871410736
test_acc: 0.8202247191011236
Experimental:
train_acc: 0.8164794007490637
test_acc: 0.8314606741573034
steady:
train_acc: 0.9812734082397003
test_acc: 0.8426966292134831
Experimental:
train_acc: 0.815230961298377
test_acc: 0.8426966292134831
steady:
train_acc: 0.982

In [8]:
print(f"avg steady train:{avg_steady_train / cut}, avg steady test:{avg_steady_test / cut}")
print(f"avg Experimental train:{avg_Experimental_train / cut}, avg Experimental test:{avg_Experimental_test / cut}")

avg steady train:0.9808988764044942, avg steady test:0.8146067415730336
avg Experimental train:0.8200998751560548, avg Experimental test:0.7865168539325843


In [9]:
keep_normal = ["Survived", "Pclass", "Sex", "SibSp", "Parch", "Fare", "Age"]


titanic_normal_df = data_pre_process(keep_normal, titanic_df)


X_normal = titanic_normal_df.drop("Survived", axis=1).values
y_normal = titanic_normal_df["Survived"].values





C:\Users\weiso131\AppData\Local\Temp\ipykernel_10052\1911100202.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if ("Age" in keep):  new_df['Age'] = imputer.fit_transform(new_df[["Age"]])
C:\Users\weiso131\AppData\Local\Temp\ipykernel_10052\1911100202.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if ("Fare" in keep): new_df['Fare'] = imputer.fit_transform(new_df[["Fare"]])


In [15]:
final_normal_model = train(X_normal, y_normal)
test(X_normal, y_normal, X_normal, y_normal, final_normal_model)

train_acc: 0.9797979797979798
test_acc: 0.9797979797979798


(0.9797979797979798, 0.9797979797979798)

In [16]:
test_path = "C:/Users/weiso131/Desktop/sklearn/titanic_disaster/titanic/test.csv"


keep_test_normal = ["Pclass", "Sex", "SibSp", "Parch", "Fare", "Age"]



titanic_test_df = pd.read_csv(test_path)

titanic_test_normal = data_pre_process(keep_test_normal, titanic_test_df).dropna()






C:\Users\weiso131\AppData\Local\Temp\ipykernel_10052\1911100202.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if ("Age" in keep):  new_df['Age'] = imputer.fit_transform(new_df[["Age"]])
C:\Users\weiso131\AppData\Local\Temp\ipykernel_10052\1911100202.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if ("Fare" in keep): new_df['Fare'] = imputer.fit_transform(new_df[["Fare"]])


In [17]:
all_prediction = final_normal_model.predict(titanic_test_normal.values)


In [18]:
submission_path = "C:/Users/weiso131/Desktop/sklearn/titanic_disaster/titanic/submission.csv"

submission_df = pd.read_csv(submission_path)

for i in range(len(submission_df)):
    index = submission_df.index[i]
    

    submission_df.at[index, "Survived"] = int(all_prediction[i])



In [19]:
print(submission_df)
submission_df.to_csv("submission_weiso.csv", index = False)

     PassengerId  Survived
0            892         0
1            893         0
2            894         1
3            895         1
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
